In [ ]:
import numpy as np
from tqdm import tqdm

## Part 1

In [ ]:
steps = [(-1,0),
         ( 1,0),
         (0,-1),
         (0, 1)]

def find_component(map_crop: np.array, pos: tuple):
    per =0
    crop = map_crop[pos]
    map_crop[pos] =0
    component = [pos]
    component_tot = [pos]
    while len(component) >0:
        block = component.pop()
        for step in steps:
            new_block = tuple(map(sum, zip(block, step)))
            if new_block not in component_tot:
                if map_crop[new_block] ==crop:
                    component.append(new_block)
                    component_tot.append(new_block)
                else:
                    per +=1
    for block in component_tot:
        map_crop[block] = 0   
    print(f"area = {len(component_tot)}, per = {per}")
    return map_crop, per*len(component_tot)

In [ ]:
size = 140+2
map_crop = [[0]*size]

with open('Day12_input.txt') as f:
     for line in f:
        map_crop.append([0] + [ord(letter)-64 for letter in line.replace('\n','') ]   +[0])
map_crop.append([0]*size)

map_crop =np.array(map_crop)
tot_count = 0
for row in tqdm(range(size)):
    for col in range(size):
        if map_crop[row][col] >0:
            map_crop, partial_count = find_component(map_crop,(row, col))
            tot_count +=partial_count
            print(partial_count)
#print(map_crop)
print(tot_count)

## Part 2

In [ ]:
steps = [(-1,0),
         ( 1,0),
         (0,-1),
         (0, 1)]

class Crop():
    def __init__(self, pos:tuple, map_crop: np.array):
        self.pos = pos
        self.crop_type = map_crop[pos]
        self.edges = []
        self.check_edges(map_crop)
        self.remove_opposite_edges()
        
    def add_edge(self, new_edge):
        if new_edge not in self.edges:
            self.edges.append(new_edge)
            
    def check_edges(self, map_crop):
        for step in steps:
            new_block = tuple(map(sum, zip(self.pos, step)))
            if map_crop[new_block] !=self.crop_type:
                self.add_edge(step)
        #print(f"to Crop {self.pos} added edges {self.edges}")
        
    def remove_opposite_edges(self):
        # if I only have 2 edges (so no |_| case ) and they are in opposite direction
        if len(self.edges) ==2 and abs(self.edges[0][0]) == abs(self.edges[1][0]):
            self.edges=[self.edges[0]]
        #    print(f"-- removed one edge, now edges = {self.edges}")

In [ ]:

def is_ver_hor_surrounded(map_crop:np.array, pos_tot:list):
    values = {map_crop[pos_tot[0]]}
    for pos in pos_tot:
        for step in steps:
            new_pos = tuple(map(sum, zip(pos, step)))
            values.add(map_crop[new_pos])
            if len(values)>2:
                return 0
            
    # removes the original values, and returns the value of the surrounding field
    values.remove(map_crop[pos_tot[0]])
    return values.pop()

In [ ]:

# I have all the coordinates and the perpendicular positions, I remove the coordinate which have no perpendicular. I take one and start a clock-wise search until I come back there, every time there is a direction change (careful with getting the right perpendiculars) I add 1 side. If there is anything left, it means it's a hole or something

steps_perp = {  (-1,0): [(0,1), (0,-1)],
                ( 1,0): [(0,1), (0,-1)],
                (0,-1): [(1,0), (-1,0)],
                (0, 1): [(1,0), (-1,0)]}

#.... 

In [ ]:

def find_component_2(map_crop: np.array, pos: tuple, new_crop_value:int):
    ext_corners =0
    crop =Crop( pos, map_crop)
    component = [crop]
    component_tot = [crop]
    pos_tot = [pos]
    
    while len(component) >0:
        block = component.pop()
        block = block.pos
        for step in steps:
            new_block = tuple(map(sum, zip(block, step)))
            if new_block not in pos_tot and map_crop[new_block] ==crop.crop_type:
                new_crop = Crop(new_block, map_crop)
                component.append(new_crop)
                component_tot.append(new_crop)
                pos_tot.append(new_block)
                    
    for block in pos_tot:
        map_crop[block] = new_crop_value
    
    
    print(f"edge coords: {[edge.pos for edge in component_tot  if len(edge.edges)>0]}")
    
    coords_borders = (separate_loops([edge.pos for edge in component_tot if len(edge.edges)>0]))
    print(f"loops = ")
    for loop in coords_borders:
        print(loop)
    
    # if we have an 1-block 
    if len(component_tot)==1:
        ext_corners = 4
    else:        
        #this computes only external corners
        for crop in component_tot:
            if crop.edges: #if it's not an internal square
                ext_corners += len(crop.edges)-1
                
    # on a 2d squared shape, internal corners = external corner -4
    sides = 2*ext_corners -4
    print(f"------- area = {len(component_tot)}, sides = {sides}")
    ID = is_ver_hor_surrounded(map_crop, pos_tot)
    surrounding_field_ID = ID 
    
    return map_crop, sides, len(component_tot), surrounding_field_ID

In [ ]:
size = 8+2
map_crop = [[0]*size]

with open('Day12_input_test.txt') as f:
     for line in f:
        map_crop.append([0] + [ord(letter)-64 for letter in line.replace('\n','') ]   +[0])
map_crop.append([0]*size)

map_crop =np.array(map_crop)
tot_count = 0
new_crop_value = 0
partials = {}

for row in tqdm(range(size)):
    for col in range(size):
        if map_crop[row][col] >0 and map_crop[row][col] <1000 :
            new_crop_value +=1000
            map_crop, sides, area, surr_field_ID = find_component_2(map_crop,(row, col), new_crop_value)
            partials[new_crop_value] = [area, sides]
            if surr_field_ID:
                partials[surr_field_ID][1] += sides

for field in partials.values():
    tot_count +=field[0]*field[1]
    print(field)
print(tot_count)
